# Assignment 4

## AI usage

## Log

## Links 

- Github: https://github.com/Satheris/IND320_SMAA
- Streamlit app: https://ind320smaa-2eg32uba6uhmrknkwtxzar.streamlit.app/

## Coding 

### Imports and system variables

In [1]:
import numpy as np
import pandas as pd 
import streamlit as st
import pymongo
from cassandra.cluster import Cluster
from pyspark.sql import SparkSession
from pyjstat import pyjstat
import requests
import json
import plotly.express as px
from pyspark import SparkConf, SparkContext

import plotly.io as pio
pio.renderers.default = "notebook+pdf"

In [2]:
# Set environment variables for PySpark (system and version dependent!) 
# if not already set persistently (e.g., in .bashrc or .bash_profile or Windows environment variables)
import os
# Set the Java home path to the one you are using ((un)comment and edit as needed):
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jre1.8.0_471"

# If you are using environments in Python, you can set the environment variables like the alternative below.
# The default Python environment is used if the variables are set to "python" (edit if needed):
os.environ["PYSPARK_PYTHON"] = "python" # or similar to "/Users/kristian/miniforge3/envs/tf_M1/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python" # or similar to "/Users/kristian/miniforge3/envs/tf_M1/bin/python"

# On Windows you need to specify where the Hadoop drivers are located (uncomment and edit if needed):
os.environ["HADOOP_HOME"] = r"C:\Users\saraa\Documents\winutils\hadoop-3.3.1"

# Set the Hadoop version to the one you are using, e.g., none:
os.environ["PYSPARK_HADOOP_VERSION"] = "without"

### Cassandra and Spark

In [6]:
# Connecting to Cassandra
# Run local Docker container first
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

WARNING	cassandra.cluster:cluster.py:__init__()- Cluster.__init__ called with contact_points specified, but no load_balancing_policy. In the next major version, this will raise an error; please specify a load-balancing policy. (contact_points = ['localhost'], lbp = None)
WARNING	cassandra.cluster:cluster.py:protocol_downgrade()- Downgrading core protocol version from 66 to 65 for ::1:9042. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version
WARNING	cassandra.cluster:cluster.py:protocol_downgrade()- Downgrading core protocol version from 65 to 5 for ::1:9042. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version


In [7]:
# Set up new keyspace
#                                              name of keyspace                        replication strategy           replication factor
session.execute("CREATE KEYSPACE IF NOT EXISTS ind320_keyspace WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")

# Create a new table
session.set_keyspace('ind320_keyspace')
# session.execute("DROP TABLE IF EXISTS ind320_keyspace.elhub_api;") # Starting from scratch every time
session.execute("CREATE TABLE IF NOT EXISTS elhub_api (ind int PRIMARY KEY, endTime text, lastUpdatedTime text, priceArea text, productionGroup text, quantityKwh float, startTime text);")

In [8]:
spark = SparkSession.builder.appName('SparkCassandraApp').\
    config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.5.1').\
    config('spark.cassandra.connection.host', 'localhost').\
    config('spark.sql.extensions', 'com.datastax.spark.connector.CassandraSparkExtensions').\
    config('spark.sql.catalog.mycatalog', 'com.datastax.spark.connector.datasource.CassandraCatalog').\
    config('spark.cassandra.connection.port', '9042').\
    config('spark.driver.host', 'localhost').\
    config('spark.driver.bindAddress', '127.0.0.1').\
    config('spark.sql.adaptive.enabled', 'true').\
        getOrCreate()

#### Testing that the connection works

In [9]:
# .load() is used to load data from Cassandra table as a Spark DataFrame
spark.read.format("org.apache.spark.sql.cassandra").options(table="my_first_table", keyspace="my_first_keyspace").load().show()

+---+--------+-------+
|ind| company|  model|
+---+--------+-------+
|  2|   Tesla|Model 3|
|  3|Polestar|      3|
|460|    Ford|Transit|
|459|    Ford| Escort|
|  1|   Tesla|Model S|
+---+--------+-------+



### MongoDB

In [10]:
def init_connection():
    return pymongo.MongoClient(st.secrets["mongo"]["uri"])

client = init_connection()

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


### Elhub API

In [11]:
URL = 'https://api.elhub.no/energy-data/v0/price-areas?dataset=PRODUCTION_PER_GROUP_MBA_HOUR' \
        '&startDate=2021-01-01T00:00:00%2B02:00&endDate=2021-02-01T00:00:00%2B02:00'

payload = { 
    "query": [], 
    "response": { "format": "json-stat2" } }

response = requests.get(URL, json=payload)
data = response.json()

# Writing the data into a file
with open(r'data\api_response_assign4.json', 'w', encoding='utf-8') as f:
    json.dump(response.json(), f, indent=2, ensure_ascii=False)
print("Response saved to 'api_response_assign4.json'")


# Prints for status
print("\nStatus Code:", response.status_code)
print("Headers:", response.headers.get('content-type'))


# Extract all production records
all_records = []
for area in data['data']:
    records = area['attributes']['productionPerGroupMbaHour']
    for record in records:
        record['priceArea'] = area['attributes']['name']  # Add area name
        all_records.append(record)

df = pd.DataFrame(all_records)
print(f"\nCreated DataFrame with {len(df)} rows")
df.head()

Response saved to 'api_response_assign4.json'

Status Code: 200
Headers: application/json; charset=utf-8

Created DataFrame with 17856 rows


,endTime,lastUpdatedTime,priceArea,productionGroup,quantityKwh,startTime
0,2021-01-01T01:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2507716.8,2021-01-01T00:00:00+01:00
1,2021-01-01T02:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2494728.0,2021-01-01T01:00:00+01:00
2,2021-01-01T03:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2486777.5,2021-01-01T02:00:00+01:00
3,2021-01-01T04:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2461176.0,2021-01-01T03:00:00+01:00
4,2021-01-01T05:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2466969.2,2021-01-01T04:00:00+01:00


### Saving df to MongoDB

In [ ]:
# Selecting a database and a collection
database = client['project']
collection = database['data']
collection.delete_many({}) # starting fresh

# Convert DataFrame to JSON and dumping to MongoDB
df_pd = df_spark.toPandas()
json_data = df_pd.to_json(orient='records')

documents = json.loads(json_data)
try: 
    collection.insert_many(documents)
    print('Successfully inserted data to MongoDB')
except Exception as e:
    print(e)

### Stopping Spark session

In [ ]:
# Stop Spark session
try:
    spark.stop()
    print('Spark session terminated successfully')
except ConnectionRefusedError:
    print("Spark session already stopped.")
except NameError:
    print('Spark session is not defined')

### Testing for Streamlit app